# JavaScript Store Generator

## Utilities

### Global imports

In [1]:
import django
import textwrap
import os
import json
import re

### Common functions

In [2]:
def camel2snake(string):
    regex = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', string)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', regex).lower()

In [3]:
def snake2camel(string):
    components = string.split('_')
    # We capitalize the first letter of each component except the first one
    # with the 'title' method and join them together.
    return components[0] + ''.join(x.title() for x in components[1:])

In [4]:
def get_internal_type(field):
    try:
        return field.get_internal_type()
    except Exception as _:
        return field.__class__.__name__

In [5]:
def get_related_model(field):
    try:
        return field.related_model._meta.label
    except Exception as _:
        return None

In [6]:
def generateJsStore(model, destination_dir):
    # Name of the model
    model_name = model.__name__.replace("Bss", "")

    # Lookup field in Django API
    lookup_field = getattr(model, 'lookup_field', 'id')
    # for aestethics
    lookup_type = 'int' if lookup_field == 'id' else 'string'
    lookup_type = f'{{{lookup_type}}}'

    file_name = camel2snake(model_name).replace('_','-')
    api_object = camel2snake(model_name).upper()
    object_name = camel2snake(model_name)

    ACTIONS = [
        'CREATE',
        'READ',
        'UPDATE',
        'DELETE',
        'LIST',
        'SCHEMA',
    ]
    ACTION_COMPLEMENTS = [
        'BEGIN',
        'SUCCESS',
        'FAILURE',
        'SET_MESSAGES',
        'SET_STATUS_CODE',
    ]
    ACTION_API_CALL = {
        'CREATE': 'create',
        'READ': 'retrieve',
        'UPDATE': 'update',
        'DELETE': 'delete',
        'LIST': 'list',
        'SCHEMA': 'options',
    }

    STATES = [action.lower() for action in ACTIONS]
    STATE_COMPLEMENTS = {
        'ing': 'false',
        'Error': 'false',
        'Success': 'false',
        'Messages': 'null',
        'StatusCode': 'null',
    }

    store = f"""        import {api_object} from '@/api/{object_name.replace('_','-')}'

        import {{"""

    for action in ACTIONS:
        store += f"""
            /* {action.capitalize()+'t' if action.endswith('A') else action.capitalize().strip('e')}ing {object_name} flow */"""

        for complement in ACTION_COMPLEMENTS:
            store += f"""
            {action}_{complement},"""

        store += """
        """

    store += f"""
            /* Handle objects */
            SET_SCHEMA,
            SET_OBJECT,
            SET_OBJECT_LIST
        }} from './_processes'

        const initialState = {{
            schema: null,
            object: null,
            objects: [],
    """

    for state in STATES:
        store += f"""
            /* {state.capitalize()} state variables */"""

        for (complement, default_value) in STATE_COMPLEMENTS.items():
            if complement == 'ing':
                store += f"""
            {state+'t' if state.endswith('a') else state.strip('e')}{complement}: {default_value},"""
            else:
                store += f"""
            {state}{complement}: {default_value},"""

        store += """
        """

    store = store.rstrip()[:-1]
    store += f"""
        }}

        const actions = {{"""

    for action in ACTIONS:
        store += f"""
            {action.lower()} (
                {{ commit }},
                {{"""

        if action == 'CREATE':
            store += f"""
                    data,"""

        if action == 'READ':
            store += f"""
                    {lookup_field},
                    query = {{}},"""

        if action == 'UPDATE':
            store += f"""
                    {lookup_field},
                    data,"""

        if action == 'DELETE':
            store += f"""
                    {lookup_field},"""

        if action == 'LIST':
            store += f"""
                    query = {{}},"""

        store += f"""
                    config = {{}}
                }} = {{}}) {{
                commit({action}_BEGIN)
                commit({action}_SET_MESSAGES, null)
                commit({action}_SET_STATUS_CODE, null)
                return {api_object}
                    .{ACTION_API_CALL[action]}("""

        if action == 'CREATE':
            store += f"""data, """

        if action == 'READ':
            store += f"""{lookup_field}, query, """

        if action == 'UPDATE':
            store += f"""{lookup_field}, data, """

        if action == 'DELETE':
            store += f"""{lookup_field}, """

        if action == 'LIST':
            store += f"""query, """

        store += f"""config)
                    .then((response) => {{"""

        if action == 'LIST':
            store += f"""
                        commit(SET_OBJECT_LIST, response.data)"""
        elif action == 'SCHEMA':
            store += f"""
                        commit(SET_SCHEMA, response.data)"""
        else:
            store += f"""
                        commit(SET_OBJECT, response.data)"""

        store += f"""
                    }})
                    .then(() => {{
                        commit({action}_SUCCESS)
                    }})
                    .catch((error) => {{
                        commit({action}_FAILURE)
                        commit({action}_SET_MESSAGES, error.response.data)
                        commit({action}_SET_STATUS_CODE, error.response.status)
                    }})
            }},
        """

    store = store.rstrip()[:-1] + f"""
        }}

        const mutations = {{"""

    for action in ACTIONS:
        store += f"""
            /**
             * {action.capitalize()+'t' if action.endswith('A') else action.capitalize().strip('e')}ing {object_name} flow
             */"""

        for complement in ACTION_COMPLEMENTS:
            store += f"""
            [{action}_{complement}] (state"""

            if complement == "SET_MESSAGES":
                store += f""", messages) {{
                state.{action.lower()}Messages = messages
            }},  """

            elif complement == "SET_STATUS_CODE":
                store += f""", statusCode) {{
                state.{action.lower()}StatusCode = statusCode
            }},  """

            else:
                state_values = ['false', 'false', 'false']

                if complement == 'BEGIN':
                    state_values[0] = 'true'
                if complement == 'FAILURE':
                    state_values[1] = 'true'
                if complement == 'SUCCESS':
                    state_values[2] = 'true'

                store += f""") {{
                state.{action.lower()+'t' if action.endswith('a') else action.lower().strip('e')}ing = {state_values[0]}
                state.{action.lower()}Error = {state_values[1]}
                state.{action.lower()}Success = {state_values[2]}
            }},  """

    store += f"""
            /**
              * Handle objects
              */
            [SET_SCHEMA] (state, schema) {{
                state.schema = schema
            }},
            [SET_OBJECT] (state, object) {{
                state.object = object
            }},
            [SET_OBJECT_LIST] (state, objects) {{
                state.objects = objects
            }}
        }}

        export default {{
            namespaced: true,
            state: initialState,
            actions,
            mutations
        }}
    """

    store = textwrap.dedent(store).split('\n')
    store = [line.rstrip() for line in store]
    store = [line.replace('    ', '  ').rstrip() for line in store]
    store = '\n'.join(store)

    store_file = os.path.join(destination_dir, file_name + ".js")

    with open(store_file, 'w') as file:
        file.write(store)

    file.close()

    print(f"""import {snake2camel(object_name)} from "./{object_name.replace('_','-')}" """)


In [7]:
models = django.apps.apps.get_models(include_auto_created=False)
models = filter(lambda m: not m._meta.abstract, models)
models = dict(map(lambda m: (m.__name__.replace("Bss", ""), m), models))
models = list(map(lambda n: (n, models[n]), sorted(models.keys())))
models

for (name, model) in models:
    # generateJsStore(model, '../frontend/src/store/')
    generateJsStore(model, 'autogenerated/js/store/')

import address from "./address" 
import association from "./association" 
import attribute from "./attribute" 
import attributeTranslation from "./attribute-translation" 
import attributeValue from "./attribute-value" 
import attributeValueTranslation from "./attribute-value-translation" 
import authorizationKey from "./authorization-key" 
import bankAccount from "./bank-account" 
import banner from "./banner" 
import benefit from "./benefit" 
import benefitTranslation from "./benefit-translation" 
import brand from "./brand" 
import brandTranslation from "./brand-translation" 
import category from "./category" 
import categoryTranslation from "./category-translation" 
import checkout from "./checkout" 
import checkoutLine from "./checkout-line" 
import city from "./city" 
import cityArea from "./city-area" 
import code from "./code" 
import collection from "./collection" 
import collectionTranslation from "./collection-translation" 
import contentType from "./content-type" 
import con